In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

In [ ]:
file=glob.glob('../input/rape-statisitics-india/*csv')

In [ ]:
file

In [ ]:
crimes=pd.DataFrame()
for i in file:
    f=pd.read_csv(i,usecols=['STATE/UT','Year','Rape','DISTRICT'])
    crimes=pd.concat([crimes,f])
    

In [ ]:
crimes

In [ ]:
for i in crimes['DISTRICT'].unique():
    try:
        if 'TOTAL' in i:
            crimes.drop(crimes[crimes['DISTRICT']==i].index.values,inplace=True)
    except:
        pass

In [ ]:
crimes.dropna(inplace=True)

In [ ]:
crimes.reset_index(drop='index',inplace=True)

In [ ]:
crimes

In [ ]:
crimes[pd.isna(crimes['Year'])]

In [ ]:
crimes['STATE/UT']=crimes['STATE/UT'].apply(lambda x:x.upper())
crimes['STATE/UT']=crimes['STATE/UT'].apply(lambda x:'DELHI' if x=='DELHI UT' else x)
crimes['STATE/UT']=crimes['STATE/UT'].apply(lambda x:x.replace(' ','') if '&' in x else x)

In [ ]:
crimes['STATE/UT'].unique()

In [ ]:
crimes['Year'].unique()

## state-wise  Rape statisitcs

In [ ]:
year=crimes['Year'].unique()
for i,j  in enumerate(crimes['STATE/UT'].unique(),start=1):
    state=crimes[crimes['STATE/UT']==j]
    year_wise_crimes=state.groupby(['Year'])['Rape'].sum().reset_index()
    plt.subplots_adjust(right=3,top=20,bottom=-30)
    plt.subplot(len(crimes['STATE/UT'].unique()),3,i)
    plt.title(j)
    plt.ylabel('Rapes',size=15)
    plt.xlabel('Year',size=12)
    plt.bar(year_wise_crimes['Year'],year_wise_crimes['Rape'],color='r')
    plt.xticks(year,rotation=90)
    plt.grid(axis='y')


## Calculating the ratio of rapes in an year to rapes in the previous year

In [ ]:
state_wise_crimes=crimes.groupby(['STATE/UT','Year'])['Rape'].sum().reset_index()


In [ ]:
state_wise_crimes

In [ ]:
state_wise_crimes['rape_next_year']=state_wise_crimes['Rape'].shift(-1)

In [ ]:
state_wise_crimes.drop(state_wise_crimes.shape[0]-1,inplace=True)

In [ ]:
state_wise_crimes.reset_index(drop='index',inplace=True)

In [ ]:
state_wise_crimes['Rapes / Rapes_in_the_previous_year']=state_wise_crimes['rape_next_year']/state_wise_crimes['Rape']

In [ ]:
state_wise_crimes

In [ ]:
for st in state_wise_crimes['STATE/UT'].unique():
    state=state_wise_crimes[state_wise_crimes['STATE/UT']==st]
    if st!='WEST BENGAL':
        state_wise_crimes.drop(state.index[-1],inplace=True)
    

In [ ]:
state_wise_crimes.reset_index(drop='index',inplace=True)

In [ ]:
state_wise_crimes

###### To undesrtand the below visualisation ,let us take an example of Andhra  Pradesh.In Andhra Pradesh's year vs 'Rapes / Rapes_in_the_previous_year' line chart the value 1.15 which corresponds to the year 2002 in the x axis means that ,rape cases in 2002 is 1.15 times the rape cases in 2001 .This implies that if 'Rapes / Rapes_in_the_previous_year' that corresponds to an year is greater than 1 , the rapes in that year is greater than the number of rapes in thr previous year and If  'Rapes / Rapes_in_the_previous_year' that corresponds to an year is less than 1 ,the rapes in that year is less than the number of rapes in thr previous year

In [ ]:
yr =[i for i in range(2002,2016)]
for i,k in enumerate(state_wise_crimes['STATE/UT'].unique(),start=1):
    state=state_wise_crimes[state_wise_crimes['STATE/UT']==k]
    plt.subplots_adjust(right=3,top=20,bottom=-40)
    plt.subplot(len(crimes['STATE/UT'].unique()),3,i)
    plt.title(k)
    plt.ylabel('Rapes / Rapes_in_the_previous_year',size=13)
    plt.xlabel('year',size=13)
    plt.plot(state['Year'],state['Rapes / Rapes_in_the_previous_year'],marker='o',markerfacecolor='red')
    plt.grid(axis='y')
    if k!='TELANGANA':
        plt.xticks(state['Year'],yr,rotation=90)
    else:
        plt.xticks(state['Year']+1)
        
        
    

## National average vs state average.Here we are trying to find the difference between each state's average rapes number in an year  and national average in that year

In [ ]:
national_rape_average=crimes.groupby(crimes['Year'])['Rape'].mean().reset_index()

In [ ]:

national_rape_average

In [ ]:
state_wise_rapes_average=crimes.groupby(['STATE/UT','Year'])['Rape'].mean().reset_index()


In [ ]:
state_wise_rapes_average

In [ ]:
state_nation_difference=[]
for i,j in enumerate(state_wise_rapes_average['Rape']):
    st=state_wise_rapes_average.iloc[i:i+1]['Rape'][i]
    nt=national_rape_average[national_rape_average['Year']==state_wise_rapes_average.iloc[i:i+1]['Year'][i]][['Rape']].values[0][0]
    state_nation_difference.append(st-nt)

In [ ]:
state_wise_rapes_average['state-Nation']=state_nation_difference

In [ ]:
state_wise_rapes_average

## If the count of   ' state average - national average ' greater than zero  in a state increases it means women are less safe in that state

In [ ]:
yr =[i for i in range(2001,2016)]
for i,k in enumerate(state_wise_rapes_average['STATE/UT'].unique(),start=1):
    state=state_wise_rapes_average[state_wise_rapes_average['STATE/UT']==k]
    plt.subplots_adjust(right=3,top=20,bottom=-40)
    plt.subplot(len(crimes['STATE/UT'].unique()),3,i)
    plt.title(k)
    plt.ylabel('state average - national average(rapes)',size=13)
    plt.xlabel('year',size=13)
    plt.plot(state['Year'],state['state-Nation'],marker='o',markerfacecolor='red')
    plt.grid(axis='y')
    if k!='TELANGANA':
        plt.xticks(state['Year'],yr,rotation=90)
    else:
        plt.xticks(state['Year'])

In [ ]:
state_average_national_average=state_wise_rapes_average[state_wise_rapes_average['state-Nation']>0]

In [ ]:
state_average_national_average_greater_than_0_count=state_average_national_average['STATE/UT'].value_counts()

## All states which are listed as Indias's 10 most dangerous states for women by India today are in  "state_average_national_average_greater_than_0_count"
https://www.indiatoday.in/diu/story/india-s-10-most-dangerous-states-for-women-uttar-pradesh-rajasthan-hathras-1727981-2020-10-03

In [ ]:
# States which exceeded average_rape than national_average at least once

state_average_national_average_greater_than_0_count

## Percentage share of each state from 2001 to 2015

In [ ]:
crimes_total=crimes['Rape'].sum()

In [ ]:
crimes_total

In [ ]:
state_crime_sum=crimes.groupby(['STATE/UT'])['Rape'].sum().reset_index()

In [ ]:
state_crime_sum['Percentage']=(state_crime_sum['Rape']/crimes_total)*100

In [ ]:
state_crime_sum=state_crime_sum.sort_values(by='Percentage')

In [ ]:
state_crime_sum.reset_index(drop='index',inplace=True)

In [ ]:
state_crime_sum

## To understand the below visualization :14.5% of rapes from 2001 to 2015 happened in Madhya Pradesh

In [ ]:
plt.figure(figsize=(16,12))
pl=plt.barh(state_crime_sum['STATE/UT'],state_crime_sum['Percentage'],label='Rape percentage(2001-2015)',color='g')
plt.bar_label(pl,fmt='%.3f')
plt.xticks([i for i in range(1,16)],fontsize=8.1,color='r')
plt.legend(loc='best')

plt.show()
